In [1]:
from google.colab import drive
from os import chdir, getcwd

drive.mount('/content/gdrive')
chdir("/content/gdrive/My Drive/SSD_ADAS/ssd_keras/")
print("Working directory:", getcwd())

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
Working directory: /content/gdrive/My Drive/SSD_ADAS/ssd_keras


In [2]:
from data_generator.object_detection_2d_data_generator import DataGenerator

input_format = ["image_name", "class_id", "xmin", "xmax", "ymin", "ymax"] # order of columns in the annotation csv

# Load validation data
validation_image_dir = "../FLIR_ADAS/validation/PreviewData"
validation_annotation_csv = "../validation.csv"

validation_dataset = DataGenerator()
validation_dataset.parse_csv(
    validation_image_dir,
    validation_annotation_csv,
    input_format
)
print("{} images in validation set.".format(validation_dataset.get_dataset_size()))

# Load training data
training_image_dir = "../FLIR_ADAS/training/PreviewData"
training_annotation_csv = "../training.csv"

training_dataset = DataGenerator()
training_dataset.parse_csv(
    training_image_dir,
    training_annotation_csv,
    input_format
)
print("{} images in training set.".format(training_dataset.get_dataset_size()))

1360 images in validation set.
7860 images in training set.


In [3]:
! ls *.h5

ssd300_alpha1_newaug_epoch-174_loss-3.7961_val_loss-3.3007.h5
ssd300_alpha4_epoch-121_loss-7.7476_val_loss-6.3080.h5
ssd300_alpha4_epoch-91_loss-7.7909_val_loss-6.3012.h5
ssd300_alpha4_newaug_epoch-161_loss-7.9542_val_loss-6.3700.h5
ssd300_alpha4_singlecat_person_epoch-129_loss-10.7450_val_loss-17.1603.h5
ssd300_geometric_aug_only_epoch-30_loss-4.4751_val_loss-3.8531.h5
ssd300_geometric_aug_only_finetuning_epoch-62_loss-3.7001_val_loss-3.1833.h5
ssd300_geometric_aug_only_finetuning_epoch-82_loss-3.6181_val_loss-3.1497.h5
ssd300_noaug_epoch-11_loss-2.7133_val_loss-3.6682.h5
ssd300_pascal_07+12_epoch-05_loss-4.2953_val_loss-3.7685.h5
ssd300_test4_epoch-03_loss-3.1129_val_loss-3.8558.h5
ssd300_untrained_coco.h5


In [4]:
from keras import backend as K
from keras.models import load_model

from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_L2Normalization import L2Normalization
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_loss_function.keras_ssd_loss import SSDLoss

def load_ssd(h5_filename):
  #Clear previously loaded models
  K.clear_session()
  
  #Fill in the custom objects so that Keras can load the model
  ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)
  
  custom_objects = {
      'AnchorBoxes': AnchorBoxes,
      'L2Normalization': L2Normalization,
      'DecodeDetections': DecodeDetections,
      'compute_loss': ssd_loss.compute_loss}
  
  #Load the model
  model = load_model(h5_filename, custom_objects)
  print("Model loaded:", h5_filename)
  
  return model
  
model = load_ssd("ssd300_alpha1_newaug_epoch-174_loss-3.7961_val_loss-3.3007.h5")

Using TensorFlow backend.


Model loaded: ssd300_alpha1_newaug_epoch-174_loss-3.7961_val_loss-3.3007.h5


In [0]:
from ssd_encoder_decoder.ssd_input_encoder import SSDInputEncoder

img_height = 300
img_width = 300
n_classes = 80 # COCO classes

scales = [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05]
aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                         [1.0, 2.0, 0.5],
                         [1.0, 2.0, 0.5]]

two_boxes_for_ar1=True
steps=[8, 16, 32, 64, 100, 300]
offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
clip_boxes=False
variances=[0.1, 0.1, 0.2, 0.2]
normalize_coords=True

# The encoder constructor needs the spatial dimensions of the model's predictor layers to create the anchor boxes.
predictor_sizes = [model.get_layer('conv4_3_norm_mbox_conf').output_shape[1:3],
                   model.get_layer('fc7_mbox_conf').output_shape[1:3],
                   model.get_layer('conv6_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv7_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv8_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv9_2_mbox_conf').output_shape[1:3]]

ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_per_layer=aspect_ratios_per_layer,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    matching_type='multi',
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.5,
                                    normalize_coords=normalize_coords)

In [0]:
from data_generator.object_detection_2d_geometric_ops import Resize, ResizeRandomInterp
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels, RandomBrightness, RandomContrast
from data_generator.data_augmentation_chain_original_ssd import SSDExpand, SSDRandomCrop
from data_generator.object_detection_2d_geometric_ops import RandomFlip
from data_generator.object_detection_2d_image_boxes_validation_utils import BoxFilter

import cv2

mean_color = [128, 128, 128]

# For validation images
convert_to_3_channels = ConvertTo3Channels()
resize = Resize(height=img_height, width=img_width)

# SSD augmentation chain without photometric ops except random contrast
random_brightness = RandomBrightness()
random_contrast = RandomContrast()

expand = SSDExpand(background=mean_color)
random_crop = SSDRandomCrop()
random_flip = RandomFlip(dim='horizontal', prob=0.5)
box_filter = BoxFilter(check_overlap=False, check_min_area=False, check_degenerate=True)
resize_random_interp = ResizeRandomInterp(
    height=img_height,
    width=img_width,
    interpolation_modes=[cv2.INTER_NEAREST, cv2.INTER_LINEAR, cv2.INTER_CUBIC, cv2.INTER_AREA, cv2.INTER_LANCZOS4],
    box_filter=box_filter)

In [0]:
# Create training and validation generators
batch_size = 8

train_generator = training_dataset.generate(
    batch_size=batch_size,
    shuffle=True,
    transformations=[convert_to_3_channels, random_brightness, random_contrast, expand, random_crop, random_flip, resize_random_interp],
    label_encoder=ssd_input_encoder,
    returns={'processed_images', 'encoded_labels'},
    keep_images_without_gt=False)

val_generator = validation_dataset.generate(
    batch_size=batch_size,
    shuffle=False,
    transformations=[convert_to_3_channels, resize],
    label_encoder=ssd_input_encoder,
    returns={'processed_images', 'encoded_labels'},
    keep_images_without_gt=False)

In [0]:
# Define model callbacks.

from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TerminateOnNaN

model_checkpoint = ModelCheckpoint(
    filepath='ssd300_alpha1_newaug_epoch-{epoch:02d}_loss-{loss:.4f}_val_loss-{val_loss:.4f}.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True)

reduce_lr_on_plateau = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    verbose=1,
    min_delta=0.0001)

terminate_on_nan = TerminateOnNaN()

callbacks = [model_checkpoint,
             reduce_lr_on_plateau,
             terminate_on_nan]

In [0]:
from keras.optimizers import Adam, SGD

# Set learning rate
lr = 0.001

adam = Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
sgd = SGD()

ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

model.compile(optimizer=sgd, loss=ssd_loss.compute_loss)

In [0]:
# Train the model

from math import ceil

initial_epoch   = 174
final_epoch     = 300
steps_per_epoch = 1000

history = model.fit_generator(
    generator=train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=final_epoch,
    callbacks=callbacks,
    validation_data=val_generator,
    validation_steps=ceil(validation_dataset.get_dataset_size()/batch_size),
    initial_epoch=initial_epoch)

Epoch 175/300
1000/1000 [==============================] - 6316s 6s/step - loss: 3.9203 - val_loss: 3.3743

Epoch 00175: val_loss improved from inf to 3.37434, saving model to ssd300_alpha1_newaug_epoch-175_loss-3.9203_val_loss-3.3743.h5
Epoch 176/300
1000/1000 [==============================] - 804s 804ms/step - loss: 3.9226 - val_loss: 3.3427

Epoch 00176: val_loss improved from 3.37434 to 3.34273, saving model to ssd300_alpha1_newaug_epoch-176_loss-3.9218_val_loss-3.3427.h5
Epoch 177/300
1000/1000 [==============================] - 804s 804ms/step - loss: 3.9135 - val_loss: 3.3665

Epoch 00177: val_loss did not improve from 3.34273
Epoch 178/300
1000/1000 [==============================] - 804s 804ms/step - loss: 3.9173 - val_loss: 3.3781

Epoch 00178: val_loss did not improve from 3.34273
Epoch 179/300
1000/1000 [==============================] - 805s 805ms/step - loss: 3.9350 - val_loss: 3.3120

Epoch 00179: val_loss improved from 3.34273 to 3.31205, saving model to ssd300_alpha1_